In [ ]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [ ]:
# Import necessary libraries
import os

import cassio

# from llama_index.tools.tavily_research.base import TavilyToolSpec
from llama_index.tools.cassandra.base import CassandraDatabaseToolSpec
from llama_index.tools.cassandra.cassandra_database_wrapper import CassandraDatabase
import openai

from llama_index.agent.openai import OpenAIAgent
from llama_index.llms.openai import OpenAI

In [ ]:
cassio.init(auto=True)

session = cassio.config.resolve_session()
if not session:
    raise Exception(
        "Check environment configuration or manually configure cassio connection parameters"
    )

# Create a CassandraDatabaseToolSpec object
db = CassandraDatabase()

spec = CassandraDatabaseToolSpec(db=db)

tools = spec.to_tool_list()
for tool in tools:
    print(tool.metadata.name)
    print(tool.metadata.description)
    print(tool.metadata.fn_schema)

cassandra_db_schema
cassandra_db_schema(keyspace: str) -> List[llama_index.core.schema.Document]
Input to this tool is a keyspace name, output is a table description
            of Apache Cassandra tables.
            If the query is not correct, an error message will be returned.
            If an error is returned, report back to the user that the keyspace
            doesn't exist and stop.

        Args:
            keyspace (str): The name of the keyspace for which to return the schema.

        Returns:
            List[Document]: A list of Document objects, each containing a table description.
        
<class 'pydantic.main.cassandra_db_schema'>
cassandra_db_select_table_data
cassandra_db_select_table_data(keyspace: str, table: str, predicate: str, limit: int) -> List[llama_index.core.schema.Document]
 Tool for getting data from a table in an Apache Cassandra database.
            Use the WHERE clause to specify the predicate for the query that uses the
            primary key. 

In [ ]:
# Create the Agent with our tools
llm = OpenAI(model="gpt-4-1106-preview")
agent = OpenAIAgent.from_tools(tools, llm=llm, verbose=True)

agent.chat("What tables are in the keyspace langchain_agent_test?")
agent.chat("What is the userid for patrick@datastax.com ?")
agent.chat("What videos did user patrick@datastax.com upload?")

Added user message to memory: What tables are in the keyspace langchain_agent_test?
=== Calling Function ===
Calling function: cassandra_db_schema with args: {"keyspace":"langchain_agent_test"}
Got output: [Document(id_='ef42ca5a-be46-496b-b8f9-037106182c71', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Table Name: user_credentials\n- Keyspace: langchain_agent_test\n- Columns\n  - password (text)\n  - user_email (text)\n  - user_id (uuid)\n- Partition Keys: (user_email)\n- Clustering Keys: \n\nTable Name: user_videos\n- Keyspace: langchain_agent_test\n- Columns\n  - description (text)\n  - title (text)\n  - user_id (uuid)\n  - video_id (uuid)\n- Partition Keys: (user_id)\n- Clustering Keys: (video_id asc)\n\n\nTable Name: users\n- Keyspace: langchain_agent_test\n- Columns\n  - email (text)\n  - id (uuid)\n  - name (text)\n- Partition Keys: (id)\n- Clustering Keys: \n\n', start_char_idx=None, end_char_idx=None, text

AgentChatResponse(response='The user ID for `patrick@datastax.com` is `522b1fe2-2e36-4cef-a667-cd4237d08b89`.', sources=[ToolOutput(content='[Document(id_=\'2b477f24-5db4-454c-871c-17fb3c542719\', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="Row(user_email=\'patrick@datastax.com\', password=None, user_id=UUID(\'522b1fe2-2e36-4cef-a667-cd4237d08b89\'))", start_char_idx=None, end_char_idx=None, text_template=\'{metadata_str}\\n\\n{content}\', metadata_template=\'{key}: {value}\', metadata_seperator=\'\\n\')]', tool_name='cassandra_db_select_table_data', raw_input={'args': (), 'kwargs': {'keyspace': 'langchain_agent_test', 'table': 'user_credentials', 'predicate': "user_email = 'patrick@datastax.com'", 'limit': 1}}, raw_output=[Document(id_='2b477f24-5db4-454c-871c-17fb3c542719', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="Row(user_email='patrick